### Introdução ao LangGraph

LangGraph é uma biblioteca para construção de fluxos de execução com LLMs de forma estruturada, utilizando grafos de estados.

Ele permite criar "state machines" com transições condicionais, paralelismo, loops e memória. Cada nó pode ser uma função, agente ou LLM.

**Vantagens:**
- Controle explícito do fluxo de execução
- Suporte para ciclos e ramificações
- Fácil integração com LangChain

In [ ]:
# !pip install langgraph langchain openai --quiet

In [ ]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage

### Conceitos-Chave

- **StateGraph**: estrutura principal do grafo, define nós e transições.
- **Node**: cada nó é uma etapa do fluxo (função, LLM, agente, etc.).
- **State**: dicionário contendo os dados mantidos e passados entre nós.
- **Edges**: conexões entre os nós, podem ser condicionais.
- **Condições**: lógica que define para qual nó ir após cada execução.

**Exemplo de estado:**

```python
{
  "messages": [HumanMessage(...), AIMessage(...)],
  "step": 2,
  "user_name": "Silvan"
}

### Exemplo 1: Grafo Simples com 1 Nó

Neste exemplo básico, criamos um LangGraph com um único nó chamado `greet`. Ele apenas retorna uma mensagem com o nome informado no estado inicial.

In [ ]:
from typing_extensions import TypedDict

class SimpleState(TypedDict):
    name: str
    message: str

In [ ]:
def greet(state):
    name = state["name"]
    return {"message": f"Olá, {name}!"}

In [ ]:
builder = StateGraph(SimpleState)

builder.add_node("cumprimenta", greet)

builder.set_entry_point("cumprimenta")
builder.add_edge("cumprimenta", END)

graph = builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
graph.invoke({"name": "Silvan"})

### Exemplo 2: Grafo com Múltiplos Nós Seguidos

Aqui, processamos uma string em três etapas:
1. Remover espaços extras
2. Converter para letras maiúsculas
3. Adicionar pontuação

Cada etapa é representada por um nó no grafo. Esse exemplo demonstra como encadear transformações sequenciais com LangGraph.

In [ ]:
class TextState(TypedDict):
    text: str
    final_text: str

In [ ]:
def strip_text(state: TextState) -> TextState:
    return {"text": state["text"].strip()}

def uppercase_text(state: TextState) -> TextState:
    return {"text": state["text"].upper()}

def punctuate_text(state: TextState) -> TextState:
    return {"final_text": state["text"] + "!"}

In [ ]:
builder = StateGraph(TextState)

builder.add_node("strip", strip_text)
builder.add_node("uppercase", uppercase_text)
builder.add_node("punctuate", punctuate_text)

builder.set_entry_point("strip")
builder.add_edge("strip", "uppercase")
builder.add_edge("uppercase", "punctuate")
builder.add_edge("punctuate", END)

graph = builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
graph.invoke({"text": "  olá mundo  "})

### Exemplo 3: Grafo com tomada de decisão

Um grafo com decisão permite que a execução siga diferentes caminhos dependendo do estado atual. Isso é feito usando **arestas condicionais**, que direcionam a execução com base em uma função de decisão.

No LangGraph, usamos `add_conditional_edges()` para isso. Essa função recebe:
- o nó atual
- uma função que decide qual transição seguir
- um dicionário que mapeia as possíveis decisões para os próximos nós

Esse padrão é útil para implementar comportamentos como:
- loops com condição de parada
- fluxos que mudam dependendo da resposta do LLM
- verificações como "se resposta contém 'tchau', encerre"

**Exemplo prático:**
```python
def decidir_proximo(state):
    if state["count"] < 3:
        return "loop"
    return END

Neste exemplo, o número recebido é verificado quanto à paridade. Dependendo se é par ou ímpar, o grafo segue caminhos diferentes.

In [ ]:
class NumberState(TypedDict):
    number: int
    result: str

In [ ]:
def decide_parity(state: NumberState) -> str:
    return "even" if state["number"] % 2 == 0 else "odd"

def handle_even(state: NumberState) -> NumberState:
    return {"result": "Número é par"}

def handle_odd(state: NumberState) -> NumberState:
    return {"result": "Número é ímpar"}

In [ ]:
builder = StateGraph(NumberState)

builder.add_node("decide", lambda x: x)
builder.add_node("even", handle_even)
builder.add_node("odd", handle_odd)

builder.set_entry_point("decide")
builder.add_conditional_edges("decide", decide_parity, {"even": "even", "odd": "odd"})
builder.add_edge("even", END)
builder.add_edge("odd", END)

graph = builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
graph.invoke({"number": 7})

### Exemplo 4: Grafo com loop

O grafo repete a execução de um nó até que uma condição de parada seja satisfeita. Isso permite simular repetições ou interações.

Neste exemplo: Recebe um texto e um número `n`. A cada iteração, adiciona uma exclamação (!) ao final do texto. O loop para após `n` iterações.

In [ ]:
class LoopState(TypedDict):
    text: str
    num: int

In [ ]:
def add_exclamation(state: LoopState) -> LoopState:
    return {
        "text": state["text"] + "!",
        "num": state["num"] - 1
    }

def should_continue(state: LoopState) -> str:
    return "loop" if state["num"] > 0 else END

In [ ]:
builder = StateGraph(LoopState)

builder.add_node("loop", add_exclamation)
builder.set_entry_point("loop")
builder.add_conditional_edges("loop", should_continue, {"loop": "loop", "__end__": END})

graph = builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
graph.invoke({"text": "Oi", "num": 4})

### Conclusão e Referências

LangGraph permite construir fluxos de execução sofisticados com LLMs, combinando flexibilidade de grafos com o poder de modelos generativos.

Ele é especialmente útil quando há necessidade de:
- Controle de estados
- Condicionalidade complexa
- Interações iterativas com LLMs

**Links úteis:**
- Documentação: https://docs.langchain.com/langgraph/
- GitHub: https://github.com/langchain-ai/langgraph